### Import CMIP5 from the module and start a session

The latest ARCCSSive stable version is available in the **~access/modules**  
Anyone on raijin can access them, you do not need to be part of the ~access group

In [1]:
% module use ~access/modules
% module load pythonlib/ARCCSSive

ERROR: Line magic function `%module` not found.
ERROR: Line magic function `%module` not found.


The database location is saved in the **$CMIP5_DB** environment variable. This is defined automatically if you have loaded ARCCSSive from the ~access/modules.

In [4]:
% set_env CMIP5_DB=sqlite:////g/data1/ua6/unofficial-ESG-replica/tmp/tree/cmip5_raijin_latest.db   

env: CMIP5_DB=sqlite:////g/data1/ua6/unofficial-ESG-replica/tmp/tree/cmip5_raijin_latest.db


Import **CMIP5** from the module and use the *method* **connect()** to open a connection to the database.

In [5]:
from ARCCSSive import CMIP5
db=CMIP5.connect()

Opening a connection creates a **session object** (in this case *db*). A *session* manages all the comunication with the database and contains all the objects which you’ve loaded or associated with it during its lifespan. Every query to the database is run through the *session*.  
There are a number of helper functions for common operations:

In [ ]:
db.models()

models() return all the models recorded in the database,  
experiments(), variables(), mips() produce similar lists for each respective field 

### Perform a simple search

To perform a search you can use the **outputs(  )** function.  
**outputs(   )** is a 'shortcut' to perform a session.query on the Instances table.
The following example shows all the input arguments you can use, the order doesn't matter and you can omit any of them.  

> db.outputs( column-name='value', ... )  

will return all the rows for the Instances table in the database.

In [19]:
results=db.outputs(variable='tas',experiment='historical',mip='Amon',model='MIROC-ESM-CHEM',ensemble='r1i1p1')

You can check how many *instances* your search returned by using the *query* method **count()**

In [20]:
results.count()

1

In this case we defined every possible constraint for the table and hence we get just one instance.  
This should always be the case, if you use all the five attributes, because every *instance* is fully defined by these and each *instance* is unique. 

We can loop through the instances returned by the search and access their attributes and their *children* ( i.e. related versions and files) attributes.

In [21]:
for o in results:
    print(o.model,o.variable,o.ensemble)
    print()
    print("drstree path is " + str(o.drstree_path()))
    for v in o.versions:
        print()
        print('version', v.version)
        print('dataset-id', v.dataset_id)
        print('is_latest', v.is_latest, 'checked on', v.checked_on)
        print()
        print(v.path)
        for f in v.files:
            print(f.filename, f.tracking_id)
            print(f.md5, f.sha256)

MIROC-ESM-CHEM tas r1i1p1

drstree path is /g/data1/ua6/DRSv2/CMIP5/MIROC-ESM-CHEM/historical/mon/atmos/r1i1p1/tas/latest

version v20111004
dataset-id NA
is_latest False checked on 2016-10-08

/g/data/ua6/unofficial-ESG-replica/tmp/tree/dias-esg-nd.tkl.iis.u-tokyo.ac.jp/thredds/fileServer/esg_dataroot/outgoing/output1/MIROC/MIROC-ESM-CHEM/historical/mon/atmos/Amon/r1i1p1/v20111004/tas
tas_Amon_MIROC-ESM-CHEM_historical_r1i1p1_185001-200512.nc 27c8356f-39f4-4d66-8915-3a5f48b76318
None None

version v20120710
dataset-id cmip5.output1.MIROC.MIROC-ESM-CHEM.historical.mon.atmos.Amon.r1i1p1.v20120710|esgf-data1.diasjp.net
is_latest True checked on 2016-10-08

/g/data/ua6/unofficial-ESG-replica/tmp/tree/dias-esg-nd.tkl.iis.u-tokyo.ac.jp/thredds/fileServer/esg_dataroot/outgoing/output1/MIROC/MIROC-ESM-CHEM/historical/mon/atmos/Amon/r1i1p1/v20120710/tas
tas_Amon_MIROC-ESM-CHEM_historical_r1i1p1_185001-200512.nc 27c8356f-39f4-4d66-8915-3a5f48b76318
3459eff8fd39ff88c8b5e72f11694862 38bf378e1f11c

### Navigate through search results

Let's have a better look at **results**  

In [22]:
type(results)

sqlalchemy.orm.query.Query

**results** is a *query object* but as we saw before we can loop through it as we do with a list.  
In this particular case we have only one *instance* returned in results, but we still need to use an index to access it.

In [23]:
type(results[0])

ARCCSSive.CMIP5.Model.Instance

A useful attribute of an *instance* is **versions**, this is a list of all the versions associated to that particular instance.  
From a database point of view these are all the rows in the **Versions table** which are related to that particular instance.

In [24]:
results[0].versions

We have two versions available for this *instance*, we can loop through them and retrieve their attributes:

In [25]:
for o in results:
    for v in o.versions:
        print()
        print(v.version)
        print()
        print(v.path)


v20111004

/g/data/ua6/unofficial-ESG-replica/tmp/tree/dias-esg-nd.tkl.iis.u-tokyo.ac.jp/thredds/fileServer/esg_dataroot/outgoing/output1/MIROC/MIROC-ESM-CHEM/historical/mon/atmos/Amon/r1i1p1/v20111004/tas

v20120710

/g/data/ua6/unofficial-ESG-replica/tmp/tree/dias-esg-nd.tkl.iis.u-tokyo.ac.jp/thredds/fileServer/esg_dataroot/outgoing/output1/MIROC/MIROC-ESM-CHEM/historical/mon/atmos/Amon/r1i1p1/v20120710/tas


If we want to get only the *latest version*, we can use the **latest(  )** method of the Instance class. 

In [26]:
results[0].latest()[0].version

'v20120710'

As you might have noticed **latest(  )** returns a list of Version objects rather than only one.  
This is because there might be different copies of the same version, downloaded from different servers.  
Currently the database lists all of them so that if you used one rather than the other in the past you can still find it.  
There are plans though to keep just one copy per version to facilitate the collection management and save storage resources.  

Other methods available for the Instances table (*objects*) are:  

+ **filenames(  )**
+ **drstree_path(  )**

In [27]:
results[0].filenames()

['tas_Amon_MIROC-ESM-CHEM_historical_r1i1p1_185001-200512.nc']

In [28]:
results[0].drstree_path()

'/g/data1/ua6/DRSv2/CMIP5/MIROC-ESM-CHEM/historical/mon/atmos/r1i1p1/tas/latest'

In [ ]:
% ls -l /g/data1/ua6/DRSv2/CMIP5/CCSM4/rcp45/day/atmos/r1i1p1/tas/latest

#### !!Warning!!
In most cases you can use directly the drstee_path() method to get to the files, but it can be useful to find all the available versions.  
For example if you want to make sure that a new version hasn't been added recently, DRSv2 it is updated only once a week.  
Or if you find that the version linked by the DRSv2 is incomplete, there might be another copy of the same version.  
We hope eventually to be able to have just one copy for each version and all of them clearly defined.

### Getting deeper in the query object 

Let's check exactly what the *outputs()* function returns

In [5]:
results=db.outputs(variable='tas',experiment='historical',mip='Amon',model='MIROC5')
print(type(results))

<class 'sqlalchemy.orm.query.Query'>


In [ ]:
# results is a "query" object This means that we haven't yet retrieve any actual value from the Instance table.
# What SQLalchemy has done up to now is to generate an SQL query statement from our input arguments to pass 
# to the database. The SQL statement is executed only when we "look" at the results, 
# ie. when we use count(), or we access the returned objects.
# This why the query is always instantaneous
# try results=db.output() and see how long it takes

In [7]:
results=db.outputs()

In [10]:
results=db.outputs(variable='tas',experiment='historical',mip='Amon',model='MIROC5').all()
print(type(results))
print( results )

<class 'list'>
[<ARCCSSive.CMIP5.Model.Instance object at 0x107d74d30>, <ARCCSSive.CMIP5.Model.Instance object at 0x107d74da0>, <ARCCSSive.CMIP5.Model.Instance object at 0x107d74e10>, <ARCCSSive.CMIP5.Model.Instance object at 0x107d74e80>, <ARCCSSive.CMIP5.Model.Instance object at 0x107d74ef0>]


In [13]:
results=db.outputs(variable='tas',experiment='historical',mip='Amon',model='MIROC5',ensemble='r1i1p1').one()
print(type(results))

<class 'ARCCSSive.CMIP5.Model.Instance'>


In [14]:
results=db.outputs(variable='tas',experiment='historical',mip='Amon',model='MIROC5').one()

MultipleResultsFound: Multiple rows were found for one()

In [15]:
results=db.outputs(variable='tas',experiment='historical',mip='Amon',model='MIROC5').first()
print(type(results))

<class 'ARCCSSive.CMIP5.Model.Instance'>


### Filter search results

We can refine our results by using the SQLalchemy **filter(  )** function.    

We will use the attributes ( or columns ) of the database tables as *constraints*.  
So, first we need to import the tables definitions from ARCCSSive. 

In [6]:
from ARCCSSive.CMIP5.Model import Instance, Version, VersionFile
#print(type(Instance))

We can also import the **unique(   )** function. This function will give us all the possible values we can use to filter over a particular attribute. 

In [11]:
from ARCCSSive.CMIP5.other_functions import unique

Let's do a new query

In [31]:
results=db.outputs(variable='tas',experiment='rcp45',mip='day')
results.count()

105

We would like to filter the results by ensemble, so we will use **unique(  )** to get all the possible ensemble values.

In [32]:
ensembles=unique(results,'ensemble')
print(ensembles)

['r10i1p1', 'r11i1p1', 'r12i1p1', 'r13i1p1', 'r14i1p1', 'r1i1p1', 'r1i2p1', 'r2i1p1', 'r2i2p1', 'r3i1p1', 'r3i2p1', 'r4i1p1', 'r5i1p1', 'r6i1p1', 'r6i1p3', 'r7i1p1', 'r8i1p1', 'r9i1p1']


**unique( results, 'attribute' )** takes two inputs:  
* results is a *query object* on the Instances table, for example what is returned by the db.outputs(  ) function
* 'attribute' is a string defining a particular attribute or column of the Instances table, for example 'model'  

**unique(   )** lists all the distinct values returned by the query for that particular attribute.

Now that we know all the ensembles values, let's choose one to filter our results.

In [34]:
r6i1p1_ens=results.filter(Instance.ensemble == 'r6i1p1')
print( r6i1p1_ens.count() )
unique(r6i1p1_ens,'ensemble')

6


['r6i1p1']

We used the **==** equals operator to select all the r61i1p1 ensembles.  
If we wanted all the "r6i1p#" ensembles regardless of their physics (p) value we could have used the **like** operator.  

In [35]:
r6i1_ens=results.filter(Instance.ensemble.like('r6i1p%'))
print( r6i1_ens.count() )
unique(r6i1_ens,'ensemble')

8


['r6i1p1', 'r6i1p3']

If we want to search two variables at the same time we can leave the variable constraints out of the query inputs,  
and then use **filter** with the **in_** operator to select them.

In [36]:
results=db.outputs(ensemble='r1i1p1',experiment='rcp45',mip='day')\
.filter(Instance.variable.in_(['tasmin','tasmax']))
results.count()

73

As you can see filter can follow directly the query, i.e. the **outputs(  )** function.  
In fact, you can refine a query with how many successive filters as you want.

In [37]:
results=db.outputs(ensemble='r1i1p1',experiment='rcp45',mip='day')\
.filter(Instance.variable.in_(['tasmin','tasmax']))\
.filter(Instance.model.like('%ESM%'))
results.count()

22

### More on SQLalchemy filter and operators

In [ ]:
from sqlalchemy import and_, or_

I can keep on adding more and more filter statements or I can combine them using **and_(  )** and **or_(  )**

In [ ]:
results=db.outputs(ensemble='r1i1p1',experiment='rcp45',mip='day')\
.filter(and_(Instance.variable.in_(['tasmin','tasmax']), Instance.model.like('%ESM%')))
results.count()

In [ ]:
results=db.outputs(ensemble='r1i1p1',experiment='rcp45',mip='day')\
.filter(or_(Instance.variable.in_(['tasmin','tasmax']), Instance.model.like('%ESM%')))
results.count()

**and_(  )** returns the same results as using two filters one after the other  
Let's try to pass two constraints directly to the same filter

In [25]:
results=db.outputs(ensemble='r1i1p1',experiment='rcp45',mip='day')\
.filter(Instance.variable.in_(['tasmin','tasmax']), Instance.model.like('%ESM%'))
results.count()

22

Again we're getting the same result as if we used **and_(   )**

### Using the search results to open the files

Once we have found the instances and versions we want to use we can use their path to find the files and work with them.  
First we load numpy and the netcdf module.

In [22]:
import numpy as np
from netCDF4 import MFDataset

All you need to open a file is the location, this is stored in the Versions table in the database as *path*.  
Alternatively you can use the *drstree* path, that is returned by the Instance *drstree_path(  )* method.

Let's define a simple function that reads a variable from a file and calculate its maximum value.  
We will use MFDataset(  ) from the netcDF4 module to open all the netcdf files in the input path as one aggregated file.

In [ ]:
def var_max(var,path):
    ''' calculate max value for variable '''
    
    # MFDataset will open all netcdf files in path as one aggregated file    
    print(path+"/*.nc")
    # open the file
    nc=MFDataset(path+"/*.nc",'r')
    # read the variable from file into a numpy array
    data = nc.variables[var][:]
    # close the file
    nc.close()
    # return the maximum
    return np.max(data)

Now we perform a search, loop through the results and pass the Version path attribute to the var_max(  ) function

In [47]:
results=db.outputs(ensemble='r1i1p1',experiment='rcp45',mip='day').filter(Instance.model.like('MIROC%'))\
.filter(Instance.variable.in_(['tas','pr']))
print(results.count())

for o in results[:2]:
    var = o.variable
    for v in o.versions:
        path=str(v.path)
        varmax=var_max(var,path)
        print()
        print('Maximum value for variable %s, version %s is %d' % (var, v.version, varmax))

8
/g/data/ua6/unofficial-ESG-replica/tmp/tree/dias-esg-nd.tkl.iis.u-tokyo.ac.jp/thredds/fileServer/esg_dataroot/outgoing/output1/MIROC/MIROC-ESM/rcp45/day/atmos/day/r1i1p1/v20120710/tas/*.nc
()
Maximum value for variable tas, version v20120710 is 330
/g/data/ua6/unofficial-ESG-replica/tmp/tree/albedo2.dkrz.de/thredds/fileServer/cmip5/output1/MIROC/MIROC5/rcp45/day/atmos/day/r1i1p1/v20120710/tas/*.nc
()
Maximum value for variable tas, version v20120710 is 325


NB if you pass directly *v.path* value you get an error because the databse return unicode string, so you need to use the str(  ) function to convert to a normal string.

### How to integrate ARCCSSive in your python script

In the previous example we simply looped through the results returned by the search as they were and passed them to a function that opened the files.  
But what if we want to do something more complex?  
Let's say that we want to pass two variables to a function and do it for every model/ensemble that has both of them for a fixed experiment and mip
Mostly users would somehow loop over the drstree path, doing something like:  
> cd /g/data1/ua6/DRSv2/CMIP5  
> list all models and save in model_list
> for model in model_list:
>     list all eavailable ensembles and save in ensemble_list
>     for ensemble in ensemble_list:
>         call_function(var1_path, var2_path)

Using ARCCSSIve we can do the same using the **unique(   )** function to return the list of all available models/ensembles.  
Let's start from defining a simple function that calculates the difference bewteen the values of two variables.

In [48]:
def vars_difference(var1,path1,var2,path2):
    ''' calculate difference between the mean of two variables '''
    
    # open the files and read both variables
    nc1=MFDataset(path1+"/*.nc",'r')
    data1 = nc1.variables[var1][:]
    nc1.close()
    nc2=MFDataset(path2+"/*.nc",'r')
    data2 = nc2.variables[var2][:]
    nc2.close()
    
    # return the difference between the two means
    return np.mean(data2) - np.mean(data1)

Now let's do the another search and get tasmin and tasmax

In [9]:
results=db.outputs(ensemble='r1i1p1',experiment='rcp45',mip='Amon').filter(Instance.model.like('MIROC%'))\
.filter(Instance.variable.in_(['tasmin','tasmax']))
results.count()

8

Get the list of distinct models and ensembles using unique

In [16]:
models=unique(results,'model')
ensembles=unique(results,'ensemble')

Now we loop over the models and the ensembles, for each model-ensemble combination we call the function if we have an instance for both variables.

In [23]:
for mod in models:
    for ens in ensembles:
        # we filter twice the reuslts, using the model and ensemble values plus one the variable at the time
        tasmin_inst=results.filter(Instance.model==mod, Instance.ensemble==ens, Instance.variable=='tasmin').first()
        tasmax_inst=results.filter(Instance.model==mod, Instance.ensemble==ens, Instance.variable=='tasmax').first()
        # we check that both filters returned something and call the function if they did
        if tasmax_inst and tasmin_inst:
            tasmin_path=tasmin_inst.latest()[0].path
            tasmax_path=tasmax_inst.latest()[0].path
            diff=vars_difference('tasmin',str(tasmin_path),'tasmax',str(tasmax_path))
            print('Difference for model %s and ensemble %s is %d' % (mod, ens, diff))

Difference for model MIROC-ESM and ensemble r1i1p1 is 3.35574
Difference for model MIROC-ESM-CHEM and ensemble r1i1p1 is 3.42337
Difference for model MIROC4h and ensemble r1i1p1 is 3.8334
Difference for model MIROC5 and ensemble r1i1p1 is 3.60251


**NB** we used **first(  )** after the filter because we know we should be getting back either 1 instance or None. We cannot use **one(  )** because that would return an error if it can't find anything.  
Also we should have checked that we are using the same versions for both variables rather than just getting the latest!  

This is just an attempt to replicate the way we use drstree but when you get more familiar with the module and with SQLalchemy you can set up more sophysticated searches. For example, in this case a better way to achieve the same result would have been using the SQLalchemy **aliased(  )** function.

In [43]:
results=db.outputs(ensemble='r1i1p1',experiment='rcp45',mip='Amon').filter(Instance.model.like('MIROC%'))\
.filter(Instance.variable.in_(['tasmin','tasmax']))
results.query(Instance.variable, label('var', Instance.variable).group_by(Instance.variable)

SyntaxError: invalid syntax (<ipython-input-43-937fdc35c605>, line 2)

In [39]:
for o in results:
    print(o.model, o.ensemble, o.variable, o.versions)

(u'MIROC4h', u'r1i1p1', u'tasmax', [<ARCCSSive.CMIP5.Model.Version object at 0x7f69e028c190>, <ARCCSSive.CMIP5.Model.Version object at 0x7f69e028cc50>])
(u'MIROC4h', u'r1i1p1', u'tasmin', [<ARCCSSive.CMIP5.Model.Version object at 0x7f69e0271050>, <ARCCSSive.CMIP5.Model.Version object at 0x7f69e0271990>])


So now each item of results is composed of two instances, all the fields are in common 

In [ ]:
results=db.outputs(ensemble='r1i1p1',experiment='rcp45',mip='Amon').filter(Instance.model.like('MIROC%'))\
.filter(Instance.variable.in_(['tasmin','tasmax']))
results.count()
results=CMIP5.query(tasminInstance, tasmaxInstance).join(
      (ensemble='r1i1p1',experiment='rcp45',mip='Amon').filter(Instance.model.like('MIROC%'))\
.filter(Instance.variable.in_(['tasmin','tasmax']))
results.count()